In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import pandas as pd
import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [8]:
#house_df = pd.read_csv('data/house_mean_imputation.csv')
house_df = pd.read_csv('data/ready_to_use_dataset.csv')
house_df = house_df.drop_duplicates(['year', 'state', 'district', 'name'])
display(house_df.shape)

(9974, 20)

In [9]:
house_df['last_house_majority'].unique()

array([nan, 'D', 'R'], dtype=object)

In [ ]:
#house_df2=house_df.set_index(['year', 'state', 'district', 'name']).dropna().copy()
house_df2=house_df.dropna().copy()
house_df2_districts=house_df2[['state','district']]
house_df2=house_df2.drop('state', axis=1).drop('district', axis=1).drop('name', axis=1)
house_df2['party']=house_df2['party'].replace(['D', 'R'], [0, 1])
house_df2['president_party']=house_df2['president_party'].replace(['D', 'R'], [0, 1])
house_df2['last_house_majority']=house_df2['last_house_majority'].replace(['D', 'R'], [0, 1])
data_train, data_test=house_df2[house_df2['year']!=2018], house_df2[house_df2['year']==2018]

x_train, y_train=data_train.drop('won', axis=1), data_train['won']

x_test, y_test=data_test.drop('won', axis=1), data_test['won']
baselineLogRegr=LogisticRegressionCV(cv=5, penalty='l2').fit(x_train, y_train)

In [28]:
display(house_df[(house_df['year']==2018)&(house_df['state']=='Alabama')&(house_df['district']=='District 1')])

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories,unemployement_rate,is_presidential_year,president_can_be_re_elected,president_party,president_overall_avg_job_approval,last_D_house_seats,last_R_house_seats,last_house_majority,fundraising
9213,District 1,1.0,Bradley Byrne,R,63.3,Alabama,151150.0,1,2018,2014.0,2,3.7,0.0,1.0,R,0.395,194.0,241.0,R,1009319.45
9214,District 1,0.0,Robert Kennedy Jr.,D,36.7,Alabama,87540.0,0,2018,NaN,0,3.7,0.0,1.0,R,0.395,194.0,241.0,R,133664.39


In [12]:
#Accuracy is defined as (TP+TN)/n
def printAccuracy(y_train, y_pred_train, y_test, y_pred_test):
    print('Training Set Accuracy: \t{:.2%}'.format(np.sum(y_train == y_pred_train) / len(y_train)))
    print('Test Set Accuracy: \t{:.2%}'.format(np.sum(y_test == y_pred_test) / len(y_test)))

y_pred_train=baselineLogRegr.predict(x_train)
y_pred_test=baselineLogRegr.predict(x_test)
printAccuracy(y_train, y_pred_train, y_test, y_pred_test)
print('Amount of districts in the predictions: {:.1%} of the total'.format(len(x_test.join(house_df2_districts).groupby(['state', 'district']).count())/435))

Training Set Accuracy: 	96.15%
Test Set Accuracy: 	94.12%
Amount of districts in the predictions: 67.1% of the total


In [31]:
#Add index to predictions from X set
y_pred_train=pd.DataFrame(y_pred_train, index=x_train.index, columns=['y_pred_train'])
y_pred_test=pd.DataFrame(y_pred_test, index=x_test.index, columns=['y_pred_test'])

#Join districts data by index
house_df2_pred_districts=house_df2_districts.join(x_test)
house_df2_pred_districts.groupby(['state', 'district']).count()

is_incumbent  party  percent  votes  year  \
state         district                                                 
Alabama       District 1              1      1        1      1     1   
              District 2              1      1        1      1     1   
              District 3              1      1        1      1     1   
              District 4              1      1        1      1     1   
              District 5              1      1        1      1     1   
              District 6              1      1        1      1     1   
              District 7              0      0        0      0     0   
Arizona       District 1              1      1        1      1     1   
              District 2              1      1        1      1     1   
              District 3              1      1        1      1     1   
              District 4              0      0        0      0     0   
              District 5              0      0        0      0     0   
              District 6              1      1        1      1     1   
              District 7              1      1        1      1     1   
              District 8              1      1        1      1     1   
              District 9              1      1        1      1     1   
Arkansas      District 3              1      1        1      1     1   
              District 4              1      1        1      1     1   
California    District 1              1      1        1      1     1   
              District 10             1      1        1      1     1   
              District 11             1      1        1      1     1   
              District 12             1      1        1      1     1   
              District 13             1      1        1      1     1   
              District 14             1      1        1      1     1   
              District 15             1      1        1      1     1   
              District 16             1      1        1      1     1   
              District 17             1      1        1      1     1   
              District 18             1      1        1      1     1   
              District 19             1      1        1      1     1   
              District 2              1      1        1      1     1   
...                                 ...    ...      ...    ...   ...   
Utah          District 4              1      1        1      1     1   
Virginia      District 1              1      1        1      1     1   
              District 10             2      2        2      2     2   
              District 11             1      1        1      1     1   
              District 2              2      2        2      2     2   
              District 3              1      1        1      1     1   
              District 4              1      1        1      1     1   
              District 5              0      0        0      0     0   
              District 7              2      2        2      2     2   
              District 8              1      1        1      1     1   
              District 9              1      1        1      1     1   
Washington    District 1              1      1        1      1     1   
              District 2              1      1        1      1     1   
              District 3              1      1        1      1     1   
              District 4              1      1        1      1     1   
              District 5              1      1        1      1     1   
              District 6              1      1        1      1     1   
              District 7              1      1        1      1     1   
              District 8              1      1        1      1     1   
              District 9              1      1        1      1     1   
West Virginia District 1              1      1        1      1     1   
              District 2              1      1        1      1     1   
              District 3              1      1        1      1     